In [ ]:
from __future__ import print_function

%matplotlib inline
import fluidsim

# Tutorial: running a simulation (user perspective)

In this tutorial, I'm going to show how to run a simple simulation with a solver that solves the 2 dimensional Navier-Stokes equations. I'm also going to present some useful concepts and objects used in FluidSim.

## A minimal simulation

First, let's see what is needed to run a very simple simulation. For the initialization (with default parameters):

In [ ]:
from fluidsim.solvers.ns2d.solver import Simul

In [ ]:
params = Simul.create_default_params()

In [ ]:
params

In [ ]:
sim = Simul(params)

And then to run the simulation:

In [ ]:
sim.time_stepping.start()

In the following, we are going to understand these 4 lines of code... But first let's clean-up by deleting the result directory of this tiny example simulation:

In [ ]:
import shutil

shutil.rmtree(sim.output.path_run)

## Importing a solver

The first line imports a "Simulation" class from a "solver" module. Any solver module has to provide a class called "Simul". We have already seen that the Simul class can be imported like this:

In [ ]:
from fluidsim.solvers.ns2d.solver import Simul

but there is another convenient way to import it from a string:

In [ ]:
Simul = fluidsim.import_simul_class_from_key("ns2d")

## Create an instance of the class Parameters

The next step is to create an object ``params`` from the information contained in the class ``Simul``:

In [ ]:
params = Simul.create_default_params()

The object `params` is an instance of the class :class:`fluidsim.base.params.Parameters` (which inherits from [fluiddyn.util.paramcontainer.ParamContainer](http://fluiddyn.readthedocs.org/en/latest/generated/fluiddyn.util.paramcontainer.html)). It is usually a quite complex hierarchical object containing many attributes.  To print them, the normal way would be to use the tab-completion of Ipython, i.e. to type "`params.`" and press on the tab key. Here with Jupyter, I can not do that so I'm going to use a command that produce a list with the interesting attributes. If you don't understand this command, you should have a look at the section on [list comprehensions](https://docs.python.org/2/tutorial/datastructures.html#list-comprehensions) of the official Python tutorial):

In [ ]:
[attr for attr in dir(params) if not attr.startswith("_")]

and some useful functions (whose names all start with ``_`` in order to be hidden in Ipython and not mixed with the meaningful parameters): 

In [ ]:
[
    attr
    for attr in dir(params)
    if attr.startswith("_") and not attr.startswith("__")
]

Some of the attributes of `params` are simple Python objects and others can be other :class:`fluidsim.base.params.Parameters`:

In [ ]:
print(type(params.nu_2))
print(type(params.output))

In [ ]:
[attr for attr in dir(params.output) if not attr.startswith("_")]

We see that the object `params` contains a tree of parameters. This tree can be represented as xml code:

In [ ]:
print(params)

## Set the parameters for your simulation

The user can change any parameters

In [ ]:
params.nu_2 = 1e-3
params.forcing.enable = False

params.init_fields.type = "noise"

params.output.periods_save.spatial_means = 1.0
params.output.periods_save.spectra = 1.0
params.output.periods_save.phys_fields = 2.0

but it is impossible to create accidentally a parameter which is actually not used:

In [ ]:
try:
    params.this_param_does_not_exit = 10
except AttributeError as e:
    print("AttributeError:", e)

And you also get an explicit error message if you use a nonexistent parameter:

In [ ]:
try:
    print(params.this_param_does_not_exit)
except AttributeError as e:
    print("AttributeError:", e)

This behaviour is much safer than using a text file or a python file for the parameters. In order to discover the different parameters for a solver, create the `params` object containing the default parameters in Ipython (`params = Simul.create_default_params()`), print it and use the auto-completion (for example writting `params.` and pressing on the tab key).

## Instantiate a simulation object

The next step is to create a simulation object (an instance of the class `solver.Simul`) with the parameters in `params`:

In [ ]:
sim = Simul(params)

which initializes everything needed to run the simulation. 

The log shows the object-oriented structure of the solver.  Every task is performed by an object of a particular class.  Of course, you don't need to understand the structure of the solver to run simulations but soon it's going to be useful to understand what you do and how to interact with fluidsim objects.

The object `sim` has a limited number of attributes:

In [ ]:
[attr for attr in dir(sim) if not attr.startswith("_")]

In the tutorial [Understand how FluidSim works](tuto_dev.ipynb), we will see what are all these attributes.

The object `sim.info` is a :class:`fluiddyn.util.paramcontainer.ParamContainer` which contains all the information on the solver (in `sim.info.solver`) and on specific parameters for this simulation (in `sim.info.solver`):

In [ ]:
print(sim.info.__class__)
print([attr for attr in dir(sim.info) if not attr.startswith("_")])

In [ ]:
sim.info.solver is sim.info_solver

In [ ]:
sim.info.params is sim.params

In [ ]:
print(sim.info.solver)

We see that a solver is defined by the classes it uses for some tasks. The tutorial [Understand how FluidSim works](tuto_dev.ipynb) is meant to explain how.

## Run the simulation

We can now start the time stepping. Since ``params.time_stepping.USE_T_END is True``, it should loop until ``sim.time_stepping.t`` is equal or larger than ``params.time_stepping.t_end = 10``.

In [ ]:
sim.time_stepping.start()

## Analyze the output

Let's see what we can do with the object ``sim.output``. What are its attributes?

In [ ]:
[attr for attr in dir(sim.output) if not attr.startswith("_")]

Many of these objects (`print_stdout`, `phys_fields`, `spatial_means`, `spect_energy_budg`, `spectra`, ...) were used during the simulation to save outputs. They can also load the data and produce some simple plots. 

Let's say that it is very simple to reload an old simulation from the saved files. There are two convenient functions to do this `fluidsim.load_sim_for_plot` and `fluidsim.load_state_phys_file`:

In [ ]:
from fluidsim import load_sim_for_plot

In [ ]:
print(load_sim_for_plot.__doc__)

In [ ]:
from fluidsim import load_state_phys_file

In [ ]:
print(load_state_phys_file.__doc__)

In [ ]:
sim = load_state_phys_file(sim.output.path_run)

For example, to display the time evolution of spatially averaged quantities (here the energy, the entrophy and their dissipation rate):

In [ ]:
sim.output.spatial_means.plot()

To plot the final state:

In [ ]:
sim.output.phys_fields.plot()

And a different time:

In [ ]:
sim.output.phys_fields.plot(time=4)

We can even plot variables that are not in the state in the solver. For example, in this solver, the divergence, which should be equal to 0:

In [ ]:
sim.output.phys_fields.plot("div")

Finally we remove the directory of this example simulation...

In [ ]:
shutil.rmtree(sim.output.path_run)